<a href="https://colab.research.google.com/github/MohamedAziz15/Lip-Sync/blob/main/Lip_Sync_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load model directly
from transformers import AutoImageProcessor, AutoModelForVideoClassification

processor = AutoImageProcessor.from_pretrained("gautamtata/videomae-base-finetuned-kinetics-finetuned-lipsync-subset-1")
model = AutoModelForVideoClassification.from_pretrained("gautamtata/videomae-base-finetuned-kinetics-finetuned-lipsync-subset-1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/471 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/944 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/345M [00:00<?, ?B/s]

In [3]:
!pip install Pillow

In [5]:
!pip install librosa transformers

In [6]:
import librosa
import numpy as np

def load_audio_and_extract_features(file_path, sample_rate=16000):
    audio, _ = librosa.load(file_path, sr=sample_rate)
    mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sample_rate)
    # Additional preprocessing steps might be needed depending on your model
    return mel_spectrogram

In [7]:
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")
# model = AutoModelForAudioClassification.from_pretrained("facebook/wav2vec2-base")

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [9]:
audio_path = "/content/drive/MyDrive/Colab Notebooks/Diverge/96_E.wav"
mel_spectrogram = load_audio_and_extract_features(audio_path)

In [11]:
audio_input = feature_extractor(mel_spectrogram, return_tensors="pt")

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [12]:
audio_input

{'input_values': tensor([[-0.4770, -0.4770, -0.4770,  ..., -0.4770, -0.4770, -0.4770],
        [-0.4975, -0.4975, -0.4975,  ..., -0.4975, -0.4975, -0.4975],
        [-0.5130, -0.5130, -0.5130,  ..., -0.5130, -0.5130, -0.5130],
        ...,
        [-0.2667, -0.2667, -0.2667,  ..., -0.2667, -0.2667, -0.2667],
        [-0.2697, -0.2697, -0.2697,  ..., -0.2697, -0.2697, -0.2697],
        [-0.0869, -0.0869, -0.0869,  ..., -0.0869, -0.0869, -0.0869]])}

In [ ]:
import av
import numpy as np

from transformers import AutoImageProcessor, VideoMAEModel
from huggingface_hub import hf_hub_download

np.random.seed(0)


def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    '''
    Sample a given number of frame indices from the video.
    Args:
        clip_len (`int`): Total number of frames to sample.
        frame_sample_rate (`int`): Sample every n-th frame.
        seg_len (`int`): Maximum allowed index of sample's last frame.
    Returns:
        indices (`List[int]`): List of sampled frame indices
    '''
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices


# video clip consists of 300 frames (10 seconds at 30 FPS)
file_path = "/content/drive/MyDrive/Colab Notebooks/Diverge/96_E.mp4"
container = av.open(file_path)

# sample 16 frames
indices = sample_frame_indices(clip_len=16, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
video = read_video_pyav(container, indices)

image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")

# prepare video for the model
video_inputs = image_processor(list(video), return_tensors="pt")

# # forward pass
# outputs = model(**inputs)
# last_hidden_states = outputs.last_hidden_state
# list(last_hidden_states.shape)

In [ ]:
# Preprocess the input data
inputs = processor(video, audio_input, return_tensors="pt")

# Pass the input data to the model
outputs = model(**inputs)